# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `config.py` file using `make_config.py`. Refer to the `README.md` or `python make_config.py --help` for information.

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

In [ ]:
# Some PYTHONPATH sanity checks
try:
    import scipp
    print("scipp found")
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e

try:
    import mantid
    print("mantid found")
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e

try:
    import scippconfig
    print("config found")
except ImportError as e:
    print("config is not available. Make sure you have generated it with `make_config.py`.")
    raise e

In [ ]:
import os
import radiography
import scipp as sc
import numpy as np

In [ ]:
experiment_dir = scippconfig.script_root

data_dir = os.path.join(experiment_dir, 'radiography', 'IMAT_sample')

# Since we don't have reference data pretend this folder contains our ref
dark_field_dir = os.path.join(data_dir, "angle0")
flat_field_dir = os.path.join(data_dir, "Flats", "RB1730044", "Tomo_test", "OpenBeam_aft1")
sample_dir = os.path.join(data_dir, "angle1")

In [ ]:
raw_data = sc.Dataset()
raw_data["sample"] = radiography.fits_to_variable(sample_dir)

# Set coords based on sample dims, as reference needs to match
raw_data.coords["projection"] = sc.Variable(["projection"], unit=sc.units.dimensionless, values=np.arange(0, raw_data["sample"].shape[0]))
raw_data.coords["x"] = sc.Variable(["x"], unit=sc.units.dimensionless, values=np.arange(0, raw_data["sample"].shape[1]))
raw_data.coords["y"] = sc.Variable(["y"], unit=sc.units.dimensionless, values=np.arange(0, raw_data["sample"].shape[2]))

In [ ]:
dark_field = radiography.fits_to_variable(dark_field_dir, average_data=True)
flat_field = radiography.fits_to_variable(flat_field_dir, average_data=True)

# This blows through 32GB of memory as raw_data["sample"] is already ~17 GB
raw_data["transmission"] = (raw_data["sample"] - dark_field) / (flat_field - dark_field)

In [ ]:
# TODO WIP below
roi_x = (100, 200)
roi_y = (100, 200)

raw_data["roi_sample"] = raw_data["sample"]

raw_data.masks["sample"] = sc.Variable(['x', 'y'], shape=(raw_data.coords["x"].shape[0], raw_data.coords["y"].shape[0]),
                                       dtype=sc.dtype.bool)